## 8.3 파일/딕셔너리 시스템

In [1]:
import os
help(os)

Help on module os:

NAME
    os - OS routines for NT or Posix depending on what system we're on.

DESCRIPTION
    This exports:
      - all functions from posix or nt, e.g. unlink, stat, etc.
      - os.path is either posixpath or ntpath
      - os.name is either 'posix' or 'nt'
      - os.curdir is a string representing the current directory (always '.')
      - os.pardir is a string representing the parent directory (always '..')
      - os.sep is the (or a most common) pathname separator ('/' or '\\')
      - os.extsep is the extension separator (always '.')
      - os.altsep is the alternate pathname separator (None or '/')
      - os.pathsep is the component separator used in $PATH etc
      - os.linesep is the line separator in text files ('\r' or '\n' or '\r\n')
      - os.defpath is the default search path for executables
      - os.devnull is the file path of the null device ('/dev/null', etc.)
    
    Programs that import and use 'os' stand a better chance of being
    porta

## 8.4	파일을 열 때 발생하는 예외 다루기

In [2]:
try:
    fname = input('Enter file to read:')
    f = open(fname, 'r')
    print(f.read())
except FileNotFoundError:
    print('File', fname, 'not found. Terminating.')

Enter file to read:r.py
name = input('What is your name? ')
account_id = input('What is your ID? ')
email = input('What is your email address? ')
print(name)
print(account_id)
print(email)


In [3]:
while True:
    try:
        fname = input('Enter file name: ')
        if not fname: 				# 빈 문자열이 입력되면 종료한다.
            break
        f = open(fname) 				# 파일 열기를 시도한다.
        print(f.read())
        f.close()
        break
    except FileNotFoundError:
        print('File could not be found. Re-enter.')

Enter file name: 


In [4]:
while True:
    fname = input('Enter file name: ')
    if not fname:
        break
    try:
        f = open(fname) 				# 파일 열기를 시도한다.
    except FileNotFoundError:
        print('File could not be found. Re-enter.')
    else:
        print(f.read())
        f.close()
        break

Enter file name: 


## 8.5 'with' 키워드 사용하기

In [5]:
with open('stuff.txt', 'r') as f:
    lst = f.readlines()
    for thing in lst:
        print(thing, end='')

Supercharged Python
Supercharged Python demonstrates techniques that allow you to write faster and more powerful code.



## 8.7 텍스트 파일 작업 상세하게 알아보기

In [6]:
with open('file.txt', 'w') as f:
    f.write('To be or not to be\n')
    f.write('That is the question.\n')
    f.write('Whether tis nobler in the mind\n')
    f.write('To suffer the slings and arrows\n')

with open('file.txt', 'r') as f:
    print(f.read())

To be or not to be
That is the question.
Whether tis nobler in the mind
To suffer the slings and arrows



In [7]:
with open('file.txt', 'r') as f:
    s = ' ' 			# 빈 칸으로 초기화한다.
    while s:
        s = f.readline()
        print(s)

To be or not to be

That is the question.

Whether tis nobler in the mind

To suffer the slings and arrows




In [8]:
with open('file.txt', 'r') as f:
    s = ' ' 			# 빈 칸으로 초기화한다.
    while s:
        s = f.readline()
        s = s.rstrip('\n')
        print(s)

To be or not to be
That is the question.
Whether tis nobler in the mind
To suffer the slings and arrows



In [9]:
with open('file.txt', 'r') as f:
    str_list = f.readlines()
    for s in str_list:
        print(s, end='')

To be or not to be
That is the question.
Whether tis nobler in the mind
To suffer the slings and arrows


## 8.9 RPN 프로젝트 안에서 텍스트 읽기

### 8.9.1	The RPN Interpreter to Date

In [10]:
import re
import operator

stack = [] 		# 값을 보관하기 위한 스택

# 스캐너 객체. 각 토큰을 고립시키고, 상황에 맞는
# 다음 작업을 수행한다. 숫자 값을 스택에 넣거나(push), 
# 연산자를 찾는 경우 스택 젤 위 항목 2개의
# 연산을 수행한다.

scanner = re.Scanner([
    (r"[ \t\n]", lambda s, t: None),
    (r"-?(\d*\.)?\d+", lambda s, t:
    stack.append(float(t))),
    (r"\d+", lambda s, t: stack.append(int(t))),
    (r"[+]", lambda s, t: bin_op(operator.add)),
    (r"[-]", lambda s, t: bin_op(operator.sub)),
    (r"[*]", lambda s, t: bin_op(operator.mul)),
    (r"[/]", lambda s, t: bin_op(operator.truediv)),
    (r"[\^]", lambda s, t: bin_op(operator.pow)),
])

# 바이너리 연산자 함수. 스택에서 2개 항목을 추출(pop)하고, 
# 연산 처리 결과를 다시 스택에 넣는다(push).

def bin_op(action):
    op2, op1 = stack.pop(), stack.pop()
    stack.append(action(op1, op2))

def main():
    while True:
        input_str = input('Enter RPN line: ')
        if not input_str:
            break
        try:
            tokens, unknown = scanner.scan(input_str)
            if unknown:
                print('Unrecognized input:', unknown)
            else:
                print(str(stack[-1]))
        except IndexError:
            print('Stack underflow.')

main()

Enter RPN line: 25 4 *
100.0
Enter RPN line: 25 4 * 50.75-
49.25
Enter RPN line: 3 3* 4 4* + .5^
5.0
Enter RPN line: 


### 텍스트 파일의 RPN 읽기

In [11]:
import re
import operator

stack = [] 		# 값을 보관하기 위한 스택

# 스캐너 객체. 각 토큰을 고립시키고, 상황에 맞는
# 다음 작업을 수행한다. 숫자 값을 스택에 넣거나(push), 
# 연산자를 찾는 경우 스택 젤 위 항목 2개의
# 연산을 수행한다.
scanner = re.Scanner([
    (r"[ \t\n]", lambda s, t: None),
    (r"-?(\d*\.)?\d+", lambda s, t:
        stack.append(float(t))),
    (r"\d+", lambda s, t: stack.append(int(t))),
    (r"[+]", lambda s, t: bin_op(operator.add)),
    (r"[-]", lambda s, t: bin_op(operator.sub)),
    (r"[*]", lambda s, t: bin_op(operator.mul)),
    (r"[/]", lambda s, t: bin_op(operator.truediv)),
    (r"[\^]", lambda s, t: bin_op(operator.pow)),
])
8
# 바이너리 연산자 함수. 스택에서 2개 항목을 추출(pop)하고, 
# 연산 처리 결과를 다시 스택에 넣는다(push).

def bin_op(action):
    op2, op1 = stack.pop(), stack.pop()
    stack.append(action(op1, op2))

def main():
    a_list = open_rpn_file()
    if not a_list:
        print('Bye!')
        return

    for a_line in a_list:
        a_line = a_line.strip()
        if a_line:
            tokens, unknown = scanner.scan(a_line)
            if unknown:
                print('Unrecognized input:', unknown)
            else:
                print(str(stack[-1]))

def open_rpn_file():
    '''소스-파일-열기 함수. 주어진 파일 이름으로
    파일을 열고, 반환된 줄들을 리스트에 담는다.
    '''
    while True:
        try:
            fname = input('Enter RPN source: ')
            f = open(fname, 'r')
            if not f:
                return None
            else:
                break
        except:
            print('File not found. Re-enter.')
    a_list = f.readlines()
    return a_list

main()

Enter RPN source: rppn.txt
File not found. Re-enter.
Enter RPN source: rpn.txt
5.0
1.4142135623730951


### 8.9.3 RPN에 대입 연산자 추가하기

In [12]:
import re
import operator

# 심볼 테이블을 제공한다. 변수의 값들이
# 이 곳에 저장된다.

sym_tab = { }
8
stack = [] # 값을 보관하기 위한 스택

# 스캐너: 심볼 테이블에 저장할 항목을 추가하고,
# 심볼 테이블에 값을 집어넣을 대입을 수행한다.

scanner = re.Scanner([
    (r"[ \t\n]", lambda s, t: None),
    (r"[+-]*(\d*\.)?\d+", lambda s, t:
        stack.append(float(t))),
    (r"[a-zA-Z_][a-zA-Z_0-9]*", lambda s, t:
        stack.append(t)),
    (r"\d+", lambda s, t: stack.append(int(t))),
    (r"[+]", lambda s, t: bin_op(operator.add)),
    (r"[-]", lambda s, t: bin_op(operator.sub)),
    (r"[*]", lambda s, t: bin_op(operator.mul)),
    (r"[/]", lambda s, t: bin_op(operator.truediv)),
    (r"[\^]", lambda s, t: bin_op(operator.pow)),
    (r"[=]", lambda s, t: assign_op()),
])

def assign_op():
    '''대입 연산 함수: 이름과 값을 추출하여
    심볼 테이블 항목을 만든다. op2가 문자열이면, 심볼 테이블에서
    찾아와야 한다(look up)는 것을 기억하자.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op2) == str: 		# op2 출처가 또 다른 변수일 수도 있음!
        op2 = sym_tab[op2]
    sym_tab[op1] = op2

def bin_op(action):
    '''바이너리 연산 계산기: 피연산자가 변수 이름이면, 
    심볼 테이블에서 값을 찾아와서, 연산하기 전에 값을 교체한다.
    '''
    op2, op1 = stack.pop(), stack.pop()
    if type(op1) == str:
        op1 = sym_tab[op1]
    if type(op2) == str:
        op2 = sym_tab[op2]
    stack.append(action(op1, op2))

def main():
    a_list = open_rpn_file()
    if not a_list:
        print('Bye!')
        return

    for a_line in a_list:
        a_line = a_line.strip()
        if a_line:
            tokens, unknown = scanner.scan(a_line)
            if unknown:
                print('Unrecognized input:', unknown)
    print(str(stack[-1]))

def open_rpn_file():
    '''소스-파일-열기 함수. 주어진 파일 이름으로
    파일을 열고, 반환된 줄들을 리스트에 담는다.
    '''
    while True:
        try:
            fname = input('Enter RPN source: ')
            if not fname:
                return None
            f = open(fname, 'r')
            break
        except:
            print('File not found. Re-enter.')
    a_list = f.readlines()
    return a_list

main()

Enter RPN source: rpn2.txt
50.0


## 8.10	바이너리 직접 읽기/쓰기

In [13]:
with open('my.dat', 'wb') as f:
    f.write(b'\x01\x02\x03\x10')

In [14]:
with open('my.dat', 'rb') as f:
    bss = f.read()
    for i in bss:
        print(i, end=' ')

1 2 3 16 

## 8.11 데이터를 고정-길이 필드로 변환하기 (struct)

### 8.11.1 한번에 하나의 숫자 쓰고 읽기

In [15]:
from struct import pack, unpack, calcsize

def write_num(fname, n):
    with open(fname, 'wb') as f:
        bss = pack('h', n)
        f.write(bss)

def read_num(fname):
    with open(fname, 'rb') as f:
        bss = f.read(calcsize('h'))
        t = struct.unpack('h', bss)
        return t[0]

In [16]:
import struct
write_num('silly.dat', 125)
print(read_num('silly.dat')) 	# 숫자 125를 쓴다

125


### 8.11.2	한번에 여러 숫자 쓰고 읽기

In [17]:
from struct import pack, unpack, calcsize

def write_floats(fname, x, y, z):
    with open(fname, 'wb') as f:
        bss = pack('fff', x, y, z)
        f.write(bss)

def read_floats(fname):
    with open(fname, 'rb') as f:
        bss = f.read(calcsize('fff'))
        return unpack('fff', bss)

In [18]:
write_floats('silly.dat', 1, 2, 3.14)
x, y, z = read_floats('silly.dat')
print(x, y, z, sep=' ')

1.0 2.0 3.140000104904175


### 8.11.3	고정-길이 문자열 쓰고 읽기

In [19]:
from struct import pack, unpack, calcsize

def write_fixed_str(fname, n, s):
    with open(fname, 'wb') as f:
        bss = pack(str(n) + 's', s.encode('utf-8'))
        f.write(bss)

def read_fixed_str(fname, n):
    with open(fname, 'rb') as f:
        bss = f.read(n)
        return bss.decode('utf-8')

In [20]:
write_fixed_str('king.d', 13, "I'm Henry the VIII I am!")
print(read_fixed_str('king.d', 13))

I'm Henry the


### 8.11.4 변수-길이 문자열 쓰고 읽기

In [21]:
from struct import pack, unpack, calcsize

def write_var_str(fname, s):
    with open(fname, 'wb') as f:
        n = len(s)
        fmt = 'h' + str(n) + 's'
        bss = pack(fmt, n, s.encode('utf-8'))
        f.write(bss)

def read_var_str(fname):
    with open(fname, 'rb') as f:
        bss = f.read(calcsize('h'))
        n = unpack('h', bss)[0]
        bss = f.read(n)
        return bss.decode('utf-8')

In [22]:
write_var_str('silly.dat', "I'm Henry the VIII I am!")
print(read_var_str('silly.dat'))

I'm Henry the VIII I am!


### 8.11.5	문자열과 숫자를 함께 쓰고 읽기

In [23]:
from struct import pack, unpack, calcsize

def write_rec(fname, name, addr, rating):
    with open(fname, 'wb') as f:
        bname = name.encode('utf-8')
        baddr = addr.encode('utf-8')
        bss = pack('9s10sf', bname, baddr, rating)
        f.write(bss)

def read_rec(fname):
    with open(fname, 'rb') as f:
        bss = f.read(calcsize('9s10sf'))
        bname, baddr, rating = unpack('9s10sf', bss)
        name = bname.decode('utf-8').rstrip('\x00')
        addr = baddr.decode('utf-8').rstrip('\x00')
        return name, addr, rating

In [24]:
write_rec('goofy.dat', 'Cleo', 'Main St.', 5.0)
print(read_rec('goofy.dat'))

('Cleo', 'Main St.', 5.0)


In [25]:
bss = pack('ff9s', 1.2, 3.14, 'I\'m Henry'.
encode('utf-8'))

In [26]:
bss = pack('ff9s0f', 1.2, 3.14, 'I\'m Henry'.encode('utf-8'))

### 8.11.6	<중> 저수준 상세 : 빅 에디안 vs 리틀 에디안

In [27]:
import struct

with open('junk.dat', 'wb') as f:
    bstr = struct.pack('hhh', 1, 2, 100)
    datalen = f.write(bstr)

In [28]:
with open('junk.dat', 'rb') as f:
    bstr = f.read(struct.calcsize('hhh'))
    a, b, c = struct.unpack('hhh', bstr)
    print(a, b, c)

1 2 100


In [29]:
with open('junk.dat', 'wb') as f:
    bstr = struct.pack('hhl', 1, 2, 100)
    datalen = f.write(bstr)

with open('junk.dat', 'rb') as f:
    bstr = f.read(struct.calcsize('hhl'))
    a, b, c = struct.unpack('hhl', bstr)

In [30]:
with open('junk.dat', 'wb') as f:
    bstr = struct.pack('<hhl', 1, 2, 100)
    datalen = f.write(bstr)

## 8.12 피클링 패키지 사용하기

In [31]:
import pickle

with open('goo.dat', 'wb') as f:
    pickle.dump([1, 2, 3], f)
    pickle.dump('Hello!', f)
    pickle.dump(3.141592, f)

In [32]:
with open('goo.dat', 'rb') as f:
    a = pickle.load(f)
    b = pickle.load(f)
    c = pickle.load(f)
    print(type(a), a)
    print(type(b), b)
    print(type(c), c)

<class 'list'> [1, 2, 3]
<class 'str'> Hello!
<class 'float'> 3.141592


In [33]:
if type(a)==list:
    print('The length of a is ', a)


The length of a is  [1, 2, 3]


In [34]:
loaded = []
with open('goo.dat', 'rb') as f:
    while True:
        try:
            item = pickle.load(f)
        except EOFError:
            print('Loaded', len(loaded), 'items.')
            break
        print(type(item), item)
        loaded.append(item)

<class 'list'> [1, 2, 3]
<class 'str'> Hello!
<class 'float'> 3.141592
Loaded 3 items.


## 8.13 shelve 패키지 사용하기

In [35]:
import shelve
nums = shelve.open('numdb')
nums['pi'] = (3.14192, False)
nums['phi'] = (2.1828, False)
nums['perfect'] = (6, True)
nums.close()

In [36]:
nums = shelve.open('numdb')
for thing in nums:
    print(thing)

pi
phi
perfect


In [37]:
print(nums['pi'])

(3.14192, False)


In [38]:
nums.close()